# Struc2vec classification

In [1]:
# Load the required packages
import pandas as pd
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import json
import numpy as np

In [2]:
class_type = "normal"
classifier = "KNN"

In [3]:
# Load the data
if class_type == "normal":
    f = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/PathwayStudio_PPI_complete_directed_weighted.emb", sep = " ", skiprows = 1, header = None, index_col = 0)

In [4]:
# Load the autoencoded embeddings
if class_type == "autoencode":
    f = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/autorcode_weighted_emb.txt", sep = "\t", header = None)
    f.drop(columns = [350], inplace = True)
    f2 = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/PathwayStudio_PPI_complete_directed_weighted.emb", sep = " ", skiprows = 1, header = None, index_col = 0)
    f.index = f2.index

In [5]:
# Extend embeddings with graphlets
if class_type == "graphlet":
    f = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/PathwayStudio_PPI_complete_directed_unweighted.emb", sep = " ", skiprows = 1, header = None, index_col = 0)
    graphlets = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/EVOKE/unfiltered.txt", header = None, sep = " ", skiprows = 1)
    graphlets.drop(columns = [73], inplace = True)
    graphlets = graphlets.apply(lambda x: np.log10(x, where = x > 0))
    graphlets.columns = ["Graphlet " + str(x) for x in range(len(list(graphlets)))]
    graphlets.index = [x + 1 for x in list(graphlets.index)]
    f = f.merge(graphlets, right_index = True, left_index = True)

In [6]:
if class_type == "combi":
    f = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/autorcode_weighted_emb.txt", sep = "\t", header = None)
    f.drop(columns = [350], inplace = True)
    f2 = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/PathwayStudio_PPI_complete_directed_weighted.emb", sep = " ", skiprows = 1, header = None, index_col = 0)
    f.index = f2.index
    graphlets = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/EVOKE/unfiltered.txt", header = None, sep = " ", skiprows = 1)
    graphlets.drop(columns = [73], inplace = True)
    graphlets = graphlets.apply(lambda x: np.log10(x, where = x > 0))
    graphlets.columns = ["Graphlet " + str(x) for x in range(len(list(graphlets)))]
    graphlets.index = [x + 1 for x in list(graphlets.index)]
    f = f.merge(graphlets, right_index = True, left_index = True)

In [7]:
# Load the reference set
ref = pd.read_csv("/Users/vlietstraw/git/Post-GWAS/Input sets/Farashi/Farashi full 2000000 bp distance no pvalue filtering.csv")

# Load the mapping file
with open("/Users/vlietstraw/git/Post-GWAS/ENSEMBL_mappings.json", "r") as fp:
    ensembl_dict = json.load(fp)
ref["nodeID"] = [ensembl_dict[x] if x in ensembl_dict.keys() else None for x in ref["gene_ids"]]

In [8]:
# Drop all unmappable candidates
ref.dropna(subset = ["nodeID"], inplace = True)
ref["nodeID"] = ref["nodeID"].astype(int)

In [9]:
# Set bp distance cutoff
max_bp_distance = 500
max_bp_distance = max_bp_distance * 1000
ref = ref[ref["bp distance absolute"] <= max_bp_distance]

In [10]:
# Drop all SNPs which no longer have a positive case
pos_counts = ref.groupby("SNP ID")["Class"].sum()
ref = ref[~ref["SNP ID"].isin(pos_counts[pos_counts == 0].index)]

In [11]:
# Identify all genes which are at least once positive
positives = ref.groupby("nodeID")["Class"].sum()
positives[positives > 1] = 1

f = f.merge(positives, left_index = True, right_index = True)

## Leave SNP out classification

In [12]:
outcomes = pd.DataFrame()
train_auc_score = []
train_auc_rank = []

In [13]:
# Perform leave-SNP-out cross validation
SNPs = list(set(ref["SNP ID"]))
for snp in SNPs:
    print("Predicting candidates for " + snp + ", number " + str(SNPs.index(snp) + 1) + " out of " + str(len(SNPs)))

    f_test = f[f.index.isin(ref[ref["SNP ID"] == snp]["nodeID"])].copy()
    f_train = f[f.index.isin(ref[ref["SNP ID"] != snp]["nodeID"])].copy()
    
    train_class = f["Class"][f.index.isin(f_train.index)]
    test_class = f["Class"][f.index.isin(f_test.index)]
    
    f_test.drop(columns = ["Class"], inplace = True)
    f_train.drop(columns = ["Class"], inplace = True)
    
    if classifier == "SVM":
        clf = SVR(gamma="auto")
    if classifier == "DT":
        clf = DecisionTreeRegressor()
    if classifier == "KNN":
        clf = KNeighborsClassifier(n_neighbors = 3)
    if classifier == "LR":
        from warnings import filterwarnings
        filterwarnings('ignore')
        clf = LogisticRegression()
    if classifier == "RF":
        clf = RandomForestRegressor(n_estimators = 1000, n_jobs = -1, max_features = "sqrt", max_depth = 5)
        
    
    clf.fit(f_train, train_class)

    outcomes = pd.concat([outcomes, pd.DataFrame({"predicted" : clf.predict(f_test),
                                                    "SNP ID" : snp,
                                                    "nodeID" : f_test.index})])
    
outcomes = outcomes.merge(ref[["SNP ID", "nodeID", "Class"]], on = ["SNP ID", "nodeID"], how = "left")

Predicting candidates for rs7968852, number 1 out of 135
Predicting candidates for rs2014202, number 2 out of 135
Predicting candidates for rs6730757, number 3 out of 135
Predicting candidates for rs2569733, number 4 out of 135
Predicting candidates for rs3096702, number 5 out of 135
Predicting candidates for rs3129878, number 6 out of 135
Predicting candidates for rs6899987, number 7 out of 135
Predicting candidates for rs3135340, number 8 out of 135
Predicting candidates for rs785373, number 9 out of 135
Predicting candidates for rs6977321, number 10 out of 135
Predicting candidates for rs878987, number 11 out of 135
Predicting candidates for rs12976534, number 12 out of 135
Predicting candidates for rs12155172, number 13 out of 135
Predicting candidates for rs4325349, number 14 out of 135
Predicting candidates for rs6958572, number 15 out of 135
Predicting candidates for rs2401138, number 16 out of 135
Predicting candidates for rs10182643, number 17 out of 135
Predicting candidates 

## Leave chromosome out validation

In [14]:
outcomes2 = pd.DataFrame()
train_auc_score2 = []
train_auc_rank2 = []

In [15]:
# Perform leave-SNP-out cross validation
chromosomes = list(set(ref["chromosome"]))

for chrom in chromosomes:
    print("Predicting candidates for chromosome " + chrom)

    f_test = f[f.index.isin(ref["nodeID"][ref["chromosome"] == chrom])].copy()
    f_train = f[f.index.isin(ref["nodeID"][ref["chromosome"] != chrom])].copy()
    
    train_class = f["Class"][f.index.isin(f_train.index)]
    test_class = f["Class"][f.index.isin(f_test.index)]
    
    f_test.drop(columns = ["Class"], inplace = True)
    f_train.drop(columns = ["Class"], inplace = True)
    
    if classifier == "SVM":
        clf = SVR(gamma="auto")
    if classifier == "DT":
        clf = DecisionTreeRegressor()
    if classifier == "KNN":
        clf = KNeighborsClassifier(n_neighbors = 3)
    if classifier == "LR":
        from warnings import filterwarnings
        filterwarnings('ignore')
        clf = LogisticRegression()
    if classifier == "RF":
        clf = RandomForestRegressor(n_estimators = 1000, n_jobs = -1, max_features = "sqrt", max_depth = 5)
    
    clf.fit(np.array(f_train), np.array(train_class))

    outcomes2 = pd.concat([outcomes2, pd.DataFrame({"predicted" : clf.predict(f_test),
                                                    "Class" : test_class, 
                                                    "chromosome" : chrom,
                                                    "nodeID" : f_test.index})])

Predicting candidates for chromosome 12
Predicting candidates for chromosome 7
Predicting candidates for chromosome 16
Predicting candidates for chromosome 3
Predicting candidates for chromosome 5
Predicting candidates for chromosome 17
Predicting candidates for chromosome 2
Predicting candidates for chromosome 8
Predicting candidates for chromosome 10
Predicting candidates for chromosome 9
Predicting candidates for chromosome 19
Predicting candidates for chromosome 1
Predicting candidates for chromosome X
Predicting candidates for chromosome 14
Predicting candidates for chromosome 20
Predicting candidates for chromosome 4
Predicting candidates for chromosome 11
Predicting candidates for chromosome 22
Predicting candidates for chromosome 6


In [16]:
outcomes.to_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/Leave-SNP-Out cross validation weighted " + class_type + " " + str(max_bp_distance) + " " + classifier + ".csv", index = False)
outcomes2.to_csv("/Users/vlietstraw/git/Post-GWAS/Struc2vec/Leave-chromosome-Out cross validation weighted " + class_type + " " + str(max_bp_distance) + " " + classifier + ".csv", index = False)

## Evaluate leave-SNP-out

In [17]:
outcomes = outcomes.sort_values(["SNP ID", "predicted"], ascending = False)
outcomes["For-SNP rank"] = outcomes.groupby("SNP ID").cumcount() + 1

In [18]:
len(set(outcomes["SNP ID"]))

135

In [19]:
sum(outcomes["Class"])

144

In [20]:
import sklearn.metrics

fpr, tpr, thresholds = sklearn.metrics.roc_curve(outcomes["Class"], -outcomes["For-SNP rank"], pos_label = 1)
print(sklearn.metrics.auc(fpr, tpr) * 100)

69.56451772628243


In [21]:
# Calculate the ROC-AUC for every SNP and average the result
SNPS2 = list(set(outcomes["SNP ID"]))
aucs = []
for snp in SNPS2:
  if len(set(outcomes["Class"][outcomes["SNP ID"] == snp])) == 1:
      aucs.append(list(set(outcomes["Class"][outcomes["SNP ID"] == snp]))[0])
  else:
      fpr, tpr, thresholds = sklearn.metrics.roc_curve(outcomes["Class"][outcomes["SNP ID"] == snp], -outcomes["For-SNP rank"][outcomes["SNP ID"] == snp], pos_label = 1)
      aucs.append(sklearn.metrics.auc(fpr, tpr))
print(sum(aucs)/len(aucs))

0.5477135461816799


In [22]:
# Calculate hits @1
sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] == 1)])

29

In [23]:
# Calculate hits @3
sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] <= 3)])

60

In [24]:
# Calculate hits @5
sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] <= 5)])

83

In [25]:
# Calculate hits @10
sum(outcomes["Class"][(outcomes["Class"] == 1) & (outcomes["For-SNP rank"] <= 10)])

125

In [26]:
outcomes["For-SNP rank"][(outcomes["Class"] == 1)].mean()

6.263888888888889

In [27]:
outcomes["For-SNP rank"][outcomes["Class"] == 1].quantile(q = [0,0.25,0.5,0.75,1])

0.00     1.0
0.25     2.0
0.50     4.0
0.75     8.0
1.00    37.0
Name: For-SNP rank, dtype: float64

## Evaluate leave-chromosome-out

In [28]:
outcomes2 = outcomes2.sort_values(["chromosome", "predicted"], ascending = False)
outcomes2["For-chromosome rank"] = outcomes2.groupby("chromosome").cumcount() + 1

In [29]:
chromosomes = list(set(outcomes2["chromosome"]))
aucs = []
for chrom in chromosomes:
  fpr, tpr, thresholds = sklearn.metrics.roc_curve(outcomes2["Class"][outcomes2["chromosome"] == chrom], -outcomes2["For-chromosome rank"][outcomes2["chromosome"] == chrom], pos_label = 1)
  aucs.append(sklearn.metrics.auc(fpr, tpr))
print(sum(aucs)/len(aucs))

0.44239832541741614


In [30]:
ref = ref.merge(outcomes2[["nodeID", "predicted"]], on = "nodeID", how = "left")

In [31]:
ref = ref.sort_values(["SNP ID", "predicted"], ascending = False)
ref["For-SNP rank"] = ref.groupby("SNP ID").cumcount() + 1

In [32]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(ref["Class"], -ref["For-SNP rank"], pos_label = 1)
print(sklearn.metrics.auc(fpr, tpr) * 100)

73.60698488149467


In [33]:
# Calculate the ROC-AUC for every SNP and average the result
SNPS2 = list(set(ref["SNP ID"]))
aucs = []
for snp in SNPS2:
  if len(set(ref["Class"][ref["SNP ID"] == snp])) == 1:
      aucs.append(list(set(ref["Class"][ref["SNP ID"] == snp]))[0])
  else:
      fpr, tpr, thresholds = sklearn.metrics.roc_curve(ref["Class"][ref["SNP ID"] == snp], -ref["For-SNP rank"][ref["SNP ID"] == snp], pos_label = 1)
      aucs.append(sklearn.metrics.auc(fpr, tpr))
print(sum(aucs)/len(aucs))

0.6262381627177855


In [34]:
# Calculate hits @1
sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] == 1)])

28

In [35]:
# Calculate hits @3
sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] <= 3)])

83

In [36]:
# Calculate hits @5
sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] <= 5)])

104

In [37]:
# Calculate hits @10
sum(ref["Class"][(ref["Class"] == 1) & (ref["For-SNP rank"] <= 10)])

120

In [38]:
ref["For-SNP rank"][(ref["Class"] == 1)].mean()

6.027777777777778

In [39]:
ref["For-SNP rank"][ref["Class"] == 1].quantile(q = [0,0.25,0.5,0.75,1])

0.00     1.0
0.25     2.0
0.50     3.0
0.75     6.0
1.00    38.0
Name: For-SNP rank, dtype: float64